In [1]:
import json

# gets all the plugins' namespace urls pairs 
with open('openplugins.json') as f:
    ns_url_map = json.load(f)
ns_url_map["BrowserOp"]

'https://testplugin.feednews.com'

In [2]:
# demo
from openplugincore import OpenPlugin
op = OpenPlugin("BrowserOp")
op.functions


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


[{'name': 'transcodeWebPage',
  'description': 'Acquire precise webpage details or real-time search engine responses based on user-input content.',
  'parameters': {'type': 'object',
   'properties': {'json': {'properties': {'link': {'type': 'string',
       'description': 'This parameter takes either a URL or a non-URL string. If a URL is given, the model will engage with the designated webpage to collect or interact with its data. If a non-URL string is given, the model will handle it as a search inquiry and try to find related real-time news or information. To guarantee the best results, make sure the input is a valid URL or a succinct search query.'}},
     'type': 'object'}}}}]

In [3]:
# utils

def cleanup_log() -> None:
    with open('ns_to_function_map.log', 'r') as file:
        lines = file.readlines()
    filtered_lines = [line for line in lines if 'Attempting to load an OpenAPI 3.' not in line]
    with open('ns_to_function_map.log', 'w') as file:
        file.writelines(filtered_lines)

def is_logged(ns):
    """Check if a namespace has been logged before."""
    with open('ns_to_function_map.log', 'r') as log_file:
        logs = log_file.readlines()
        for log in logs:
            if f"<OP>{ns}</OP>" in log:
                return True
    return False

In [4]:
import os

# create ns_to_function_map.json if not exist
if not os.path.exists('ns_to_function_map.json'):
    with open('ns_to_function_map.json', 'w') as f:
        json.dump({}, f)

In [5]:
import logging
logging.basicConfig(filename='ns_to_function_map.log', level=logging.INFO)


# get ns_to_function_map
ns_to_function_map = {}
with open('ns_to_function_map.json') as f:
    ns_to_function_map = json.load(f)
cleanup_log()

for idx, ns in enumerate(ns_url_map):
    # if ns in logging then skip
    if is_logged(ns):
        continue
    logging.info(f"{idx}/<OP>{ns}</OP>")

    # save to json, cleanup log, and print progress
    if idx % 10 == 0:
        print(f"{idx}/{len(ns_url_map)}")
        cleanup_log()
        with open('ns_to_function_map.json', 'w') as f:
            json.dump(ns_to_function_map, f)

    # try and map the functions
    try:
        op = OpenPlugin(ns)
        if not op.functions:
            raise Exception("No functions")
        ns_to_function_map[ns] = op.functions
    except Exception as e:
        print("\n\nError: ", ns, "\n", e)
        pass

# save to json, cleanup log, and print progress
print(f"{idx}/{len(ns_url_map)}")
cleanup_log()
# save ns_to_function_map
with open('ns_to_function_map.json', 'w') as f:
    json.dump(ns_to_function_map, f)


880/997
890/997


Error:  Astrology 
 503 Server Error: Service Unavailable for url: https://astrology.divinationzone.com/


Error:  ATSResumeCopilot 
 HTTPSConnectionPool(host='dbea2352-8fb7-43e9-a230-8f49987ba84d.plugin.webbot.chat', port=443): Max retries exceeded with url: /.well-known/ai-plugin.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001ADB1F8C700>: Failed to resolve 'dbea2352-8fb7-43e9-a230-8f49987ba84d.plugin.webbot.chat' ([Errno 11001] getaddrinfo failed)"))


Error:  emails 
 403 Client Error: Forbidden for url: https://email-marketing-plugin-production.b12.io/.well-known/ai-plugin.json


Error:  scheduling 
 403 Client Error: Forbidden for url: https://scheduling-plugin-production.b12.io/.well-known/ai-plugin.json


Error:  canar_ai_job_agent 
 HTTPSConnectionPool(host='chat.gptplugin.decanaria.com', port=443): Max retries exceeded with url: /.well-known/ai-plugin.json (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_